In [2]:
import pandas as pd
df=pd.read_csv("data.csv")
df

,Unnamed: 0,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Fake
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",True
2,2,Why the Truth Might Get You Fired,Fake
3,3,15 Civilians Killed In Single US Airstrike Hav...,Fake
4,4,Iranian woman jailed for fictional unpublished...,Fake
...,...,...,...
34319,34319,trump vs clinton risk vs disaster,Real
34320,34320,gingrich slutshames megyn kelly,Real
34321,34321,youtube bans clintons black son,Real
34322,34322,wikileaks bombshells hillary need know,Real


In [3]:
df.shape

(34324, 3)

In [5]:
df = df.drop(["Unnamed: 0"],axis = 1)

In [6]:
df.shape

(34324, 2)

In [9]:
df.label.value_counts()

True    20183
Fake    14141
Name: label, dtype: int64

In [8]:
df["label"] = df["label"].replace("Real", "True")

In [12]:
df["label_number"] = df["label"].map({"Fake" : 0 , "True" : 1})

In [13]:
df

,text,label,label_number
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Fake,0
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",True,1
2,Why the Truth Might Get You Fired,Fake,0
3,15 Civilians Killed In Single US Airstrike Hav...,Fake,0
4,Iranian woman jailed for fictional unpublished...,Fake,0
...,...,...,...
34319,trump vs clinton risk vs disaster,True,1
34320,gingrich slutshames megyn kelly,True,1
34321,youtube bans clintons black son,True,1
34322,wikileaks bombshells hillary need know,True,1


In [14]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [15]:
df["vector"] = df["text"].apply(lambda x : nlp(x).vector)

In [20]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(df["vector"].values ,df["label_number"],test_size= 0.2, random_state = 2023)

In [21]:
X_train.shape ,X_test.shape

((27459,), (6865,))

In [24]:
X_train# need to chanr 2D array for model building by numpy stack function

array([array([-1.7373939 , -1.2899987 ,  2.432154  ,  1.5913635 ,  2.43435   ,
              -0.7408502 , -1.4814276 , -1.6324509 ,  2.4812338 , -1.2418287 ,
               2.2023163 ,  0.43932128, -1.5108887 ,  1.5895138 ,  1.5394651 ,
               0.6053438 , -0.7224763 ,  2.016825  ,  0.19954874,  2.7636976 ,
               0.6350405 ,  0.22601743, -2.03314   ,  2.189602  ,  2.3258739 ,
               2.898434  , -3.4433625 ,  1.8756146 , -0.8542662 ,  3.323076  ,
               1.2476175 , -0.304318  ,  0.4143113 ,  1.0195563 , -5.571975  ,
              -0.01752883, -1.0021137 , -0.58361495, -3.408975  , -1.4624447 ,
               2.353005  ,  0.05999002,  0.26496005, -0.14003524, -0.72712994,
              -1.6742188 , -2.5468388 , -0.66312015,  1.4628025 , -0.476655  ,
              -2.3059545 , -0.8368801 ,  2.2244842 , -2.178565  , -0.58994687,
               1.5790449 , -2.820065  , -2.8261025 , -0.06143006, -0.445755  ,
              -4.424521  , -1.8882717 , -1.3315792 ,

In [25]:
import numpy as np

In [26]:
X_train_2d = np.stack(X_train)
X_test_2d = np.stack(X_test)

In [27]:
X_train_2d

array([[-1.7373939 , -1.2899987 ,  2.432154  , ..., -1.0682462 ,
         0.39538503,  0.23253378],
       [-0.74950755,  0.040088  , -3.6972876 , ..., -0.01734251,
        -2.0425925 ,  0.06658497],
       [-3.8631856 , -1.7476883 , -1.0251956 , ..., -2.5566847 ,
         0.5715999 , -2.2874186 ],
       ...,
       [-0.71433324, -2.6546226 ,  1.0789267 , ..., -1.7536501 ,
         0.5129281 , -1.3125262 ],
       [-0.6748252 ,  0.3259771 ,  0.25072592, ..., -2.1525114 ,
        -2.7194507 , -0.97084665],
       [-2.4761105 , -1.8817599 ,  0.11564774, ..., -2.0783167 ,
        -0.3008966 ,  1.2459223 ]], dtype=float32)

In [30]:
from sklearn.naive_bayes import MultinomialNB

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_train_embed = scaler.fit_transform(X_train_2d)
scaled_test_embed = scaler.fit_transform(X_test_2d)

In [33]:
clf = MultinomialNB()
clf.fit(scaled_train_embed,y_train)

MultinomialNB()

In [34]:
y_pred = clf.predict(scaled_test_embed)

In [36]:
from sklearn.metrics import classification_report

In [37]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.87      0.51      0.65      2791
           1       0.74      0.95      0.83      4074

    accuracy                           0.77      6865
   macro avg       0.81      0.73      0.74      6865
weighted avg       0.79      0.77      0.76      6865



In [46]:
from sklearn.neighbors import KNeighborsClassifier


clf = KNeighborsClassifier(n_neighbors=10,metric="euclidean")
clf.fit(scaled_tra,y_train)



KNeighborsClassifier(metric='euclidean', n_neighbors=10)

In [47]:
y_pred = clf.predict(scaled_test_embed)
print(classification_report(y_test,y_pred))


              precision    recall  f1-score   support

           0       0.41      1.00      0.58      2791
           1       0.00      0.00      0.00      4074

    accuracy                           0.41      6865
   macro avg       0.20      0.50      0.29      6865
weighted avg       0.17      0.41      0.24      6865



C:\Users\Nitish Thakur\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Nitish Thakur\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Nitish Thakur\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
